In [1]:
import numpy as np

Let's consider the prediction problem from Q4 of HW1. We are given an input $x \in \{0,1\}$, and we are trying to predict a label $t \in \{0, 1\}$. For this problem, we assumed that the data generating distribution $p_{data}$ was any distribution such that $p_{data}(x) > 0$ for all $x \in \{0,1\}$. Let's consider a special case of $p_{data}$, which is given by the following table.

 &nbsp; | `x = 0` | `x = 1`
--- | --- | ---
**`t = 0`** | 9/20 | 2/20
**`t = 1`** | 6/20 | 3/20

We assume that our data is random pairs $(x, t) \sim p_{data}(x,t)$.



In [2]:
def data_gen_joint(x, t):
  """Returns the data generating joint distribution p_data(x,t)."""
  if x == 0 and t == 0:
    return 9/20
  elif x == 0 and t == 1:
    return 6/20
  elif x == 1 and t == 0:
    return 2/20
  elif x == 1 and t == 1:
    return 3/20

Based on this joint distribution, we can compute the relevant marginal $p_{data}(x)$ and conditional $p_{data}(t | x)$.

In [3]:
def data_gen_marginal(x):
  """Returns the data generating marginal distribution p_data(x).
  
  p_data(x) = sum_t p_data(x, t)
  """
  px = 0
  for t in [0, 1]:
    px = px + data_gen_joint(x, t)
  return px

def data_gen_conditional(t, x):
  """Returns the data generating conditional distribution p_data(t | x).
  
    p_data(t | x) = p_data(x,t) / p_data(x)
  """
  return data_gen_joint(x, t) / data_gen_marginal(x)

Let us consider predictors, which are functions $y : \{0, 1\} \to \{0, 1\}$. It is not too hard to convince yourself that there are *only* 4 possible predictors:

 &nbsp; &nbsp; &nbsp; &nbsp;| $y_i(0)$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; |$y_i(1)$   &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; 
--- | --- | ---
$y_1$ | 0 | 1
$y_2$ | 1 | 0
$y_3$ | 0 | 0
$y_4$ | 1 | 1


In [12]:
def y_1(x):
  return x

def y_2(x):
  return 1 - x

def y_3(x):
  return 0

def y_4(x):
  return 1


def y_star(x):
    """
    This computes the optimal predictor
    
    y*(x) = arg max_{t in {0, 1}} p_data(t | x)
    """
    # Recall that t is either 0 or 1
    if data_gen_conditional(0, x) > data_gen_conditional(1, x):
        return 0
    else:
        return 1

all_predictors = [y_1, y_2, y_3, y_4]

for i, y in enumerate(all_predictors):
    print(f"y_{i+1}: y_{i+1}(0)={y(0)} y_{i+1}(1)={y(1)}")
    
print(f"y_star: y_star(0)={y_star(0)} y_star(1)={y_star(1)}")

y_1: y_1(0)=0 y_1(1)=1
y_2: y_2(0)=1 y_2(1)=0
y_3: y_3(0)=0 y_3(1)=0
y_4: y_4(0)=1 y_4(1)=1
y_star: y_star(0)=0 y_star(1)=1


We will score predictors based on the 0-1 loss:

$$L(y, t) = \begin{cases} 0 & \text{if } y = t\\1 &\text{if } y \neq t\end{cases}$$

Given a predictor $y$, we can compute the expected loss on the data generating distribuition:

$$\mathcal{R}[y] = \mathbb{E}_{(X,T) \sim p_{data}}[L(y(X), T)] = \sum_{t \in \{0,1\}} \sum_{x \in \{0,1\}} p_{data}(x,t) L(y(x), t)$$

Given a predictor $y$ and a data set $\mathcal{D}$ with $N$ data points, we can compute the average loss:

$$\hat{\mathcal{R}}[y, \mathcal{D}] = \sum_{(x^{(i)}, t^{(i)}) \in \mathcal{D}} \frac{1}{N} L(y(x^{(i)}), t^{(i)})$$


In [13]:
def loss(y, t):
  """The 0-1 loss function."""
  return int(y != t)

def expected_loss(y, p_data):
  """Compute the expected loss of y : {0,1} -> {0,1}."""
  L = 0
  for t in [0, 1]:
    for x in [0, 1]:
      L = L + p_data(x, t) * loss(y(x), t)
  return L

def average_loss(y, D):
  """Compute the average loss of y : {0,1} -> {0,1}."""
  N = len(D)
  L = 0
  for (xi, ti) in D:
    L = L + loss(y(xi), ti) 
  return L / N

Let's evaluate the predictors $y_i$ on expected loss $\mathcal{R}[y_i]$ under the data generating distribution.

In [6]:
predictor_losses = [expected_loss(y, data_gen_joint) for y in all_predictors]
for i, predictor_loss in enumerate(predictor_losses):
  print(f"R[y_{i+1}]=" + "{:.2f}".format(predictor_loss))

# Report
print(f"Which is the optimal predictor in terms of expected loss? y_{np.argmin(predictor_losses)+1}")

R[y_1]=0.40
R[y_2]=0.60
R[y_3]=0.45
R[y_4]=0.55
Which is the optimal predictor in terms of expected loss? y_1


Notice that there is a *unique* optimal predictor: $y_1$. This predictor is the same as $y^{\star}$. Let's consider how hard it is to *learn* this predictor from a finite training set. First, let's write a function for generating data sets.


In [7]:
def sample_data_set(N):
  """Sample a data set according to the data generating distribution"""
  D = []
  for i in range(N):
    # First, sample x.
    # px = [p(X=0), p(X=1)]
    px = [data_gen_marginal(0), data_gen_marginal(1)]
    x = np.random.choice([0, 1], p=px)

    # Now we sample t given x.
    # pt_given_x = [p(T=0 | X=x), p(T=1 | X=x)]
    pt_given_x = [data_gen_conditional(0, x), data_gen_conditional(1, x)]
    t = np.random.choice([0,1], p=pt_given_x)

    # Add (x,t) to training set
    D.append((x, t))
  return D

Let's sample a finite training set $\mathcal{D}^{train}$, evaluate all of the predictors in terms of average loss $\hat{\mathcal{R}}[y_i, \mathcal{D}^{train}]$ on the training set and pick the best one $\hat{y}^{\star}$. Which one did we pick?

In [28]:
N = 1000
train_set = sample_data_set(N)
print("Training set")
for i, (x, t) in enumerate(train_set):
  print(f"x_{i}: {x}  t_{i}: {t}")

print("\nLet's evaluate the predictors on average training loss.")
predictor_losses = [average_loss(y, train_set) for y in all_predictors]
for i, predictor_loss in enumerate(predictor_losses):
  print(f"y_{i+1} average training loss: " + "{:.2f}".format(predictor_loss))

# This code finds the predictor with the minimum training loss
which_predictor = np.argmin(predictor_losses)
best_predictor = all_predictors[which_predictor]

# Report
print(f"\nWhich is an optimal predictor in terms of average training loss? y_{which_predictor+1}")
print("What expected loss does it get? {:.2f}".format(expected_loss(best_predictor, data_gen_joint)))

Training set
x_0: 0  t_0: 0
x_1: 1  t_1: 0
x_2: 0  t_2: 1
x_3: 1  t_3: 0
x_4: 0  t_4: 0
x_5: 1  t_5: 1
x_6: 1  t_6: 1
x_7: 0  t_7: 0
x_8: 0  t_8: 0
x_9: 1  t_9: 0
x_10: 1  t_10: 0
x_11: 0  t_11: 1
x_12: 0  t_12: 0
x_13: 0  t_13: 0
x_14: 1  t_14: 0
x_15: 1  t_15: 1
x_16: 0  t_16: 1
x_17: 0  t_17: 1
x_18: 0  t_18: 1
x_19: 0  t_19: 0
x_20: 0  t_20: 1
x_21: 1  t_21: 1
x_22: 1  t_22: 1
x_23: 1  t_23: 1
x_24: 0  t_24: 0
x_25: 0  t_25: 1
x_26: 1  t_26: 0
x_27: 0  t_27: 0
x_28: 1  t_28: 1
x_29: 0  t_29: 1
x_30: 0  t_30: 0
x_31: 0  t_31: 1
x_32: 1  t_32: 1
x_33: 0  t_33: 0
x_34: 0  t_34: 0
x_35: 0  t_35: 0
x_36: 0  t_36: 0
x_37: 0  t_37: 0
x_38: 0  t_38: 1
x_39: 0  t_39: 0
x_40: 1  t_40: 1
x_41: 0  t_41: 1
x_42: 0  t_42: 0
x_43: 0  t_43: 1
x_44: 0  t_44: 0
x_45: 0  t_45: 1
x_46: 0  t_46: 0
x_47: 0  t_47: 0
x_48: 0  t_48: 0
x_49: 0  t_49: 0
x_50: 0  t_50: 1
x_51: 0  t_51: 1
x_52: 0  t_52: 1
x_53: 1  t_53: 0
x_54: 0  t_54: 0
x_55: 0  t_55: 0
x_56: 1  t_56: 0
x_57: 0  t_57: 0
x_58: 1  t_58: 1
x_59

If you run the previous cell a few times, you'll see that the best predictor $\hat{y}^{\star}$ is *random*, because our training set was *random*. Let's automate this process and see *how often* we pick the optimal predictor $\hat{y}^{\star} = y_1$ from random training sets.

In [9]:
def pick_predictor(train_set):
  """Pick the best predictor on this train set."""
  predictor_losses = [average_loss(y, train_set) for y in all_predictors]
  i = np.argmin(predictor_losses)
  return all_predictors[i], i

With the code below you can vary the size of training sets $N$ and the number of random training sets that we sample $m$ to estimate the probability of picking the optimal predictor. Notice, as $N \to \infty$, we end up always picking $y_1$.

In [20]:
N = 100
m = 100

how_often_optimal = 0
for j in range(m):
  # Sampling a new training set of size N
  train_set = sample_data_set(N)

  # Pick the best predictor
  predictor, which_predictor = pick_predictor(train_set)
  how_often_optimal += (which_predictor + 1 == 1)
    
  # Report
  print((f"Best predictor on train set {j} is " +
        f"y_{which_predictor+1} and it has expected loss R[y_{which_predictor+1}]=" + 
        "{:.2f}".format(expected_loss(predictor, data_gen_joint))))

# Report
print((f"\nOver {m} training sets of size {N}, "+
       f"we picked the optimal predictor y_1 {how_often_optimal/m * 100} percent of the time"))

Best predictor on train set 0 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 1 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 2 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 3 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 4 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 5 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 6 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 7 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 8 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 9 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 10 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 11 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 12 is y_1 and it has expected loss R[y_1]=0.40
Best predictor on train set 13 is y

Finally, let's see how good of an approximation we can form of the expected loss $\hat{\mathcal{R}}[\hat{y}^{\star}, \mathcal{D}^{test}] \approx \mathcal{R}[\hat{y}^{\star}]$ using test sets.

In [27]:
train_N = 1
test_N = 1000
test_set = sample_data_set(test_N)

# Sampling a new training set of size N
train_set = sample_data_set(train_N)

# Pick the best predictor
predictor, which_predictor = pick_predictor(train_set)

# Report
print((f"Best training set predictor y_{which_predictor+1} "+
       "has expected loss {:.2f} and ".format(expected_loss(predictor, data_gen_joint)) +
       "average test loss {:.2f} on test set of size {} and ".format(average_loss(predictor, test_set), test_N) +
       "average training loss {:.2f} on test set of size {}.".format(average_loss(predictor, train_set), train_N)
      ))

Best training set predictor y_2 has expected loss 0.60 and average test loss 0.61 on test set of size 1000 and average training loss 0.00 on test set of size 1.
